# OpenAI - Chat completions

API into ChatGPT

Chat is a series of messages and each message has a role (system, user or assistant) and a content.

Parameters:

- Model (gpt3.5, gpt-4...)

- Temperature or top_p (how random the responses are)

- N (number of choices to generate / mutiple responses)

- Stream (whether to stram out the output one word at a time)

- Stop (Stop sequences)

- Max_tokens ("to controll budget")

- Presence_penalty (penalize or encourage tokens that already appeared)

- Logit_bias (map to directly affect the probability of certain tokens)

In [1]:
import os
from openai import OpenAI

In [4]:
client = OpenAI(api_key="") #my api key

In [ ]:
prompt = input("Enter your prompt: ")

completion = client.chat.completions.create(
    model= "gpt-3.5-turbo",
    messages=[
        {'role': "system",
         'content': "You are a helpful assistant.",
         'role': "user",
         'content': prompt}
    ]
)

print("\n" + completion.choices[0].message.content)

print("\nComplete response message:\n")
print(completion)

## Functions or tools

In [ ]:
import json

In [ ]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)


def run_conversation(prompt):
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": prompt}]
    tools = [ #function
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        tools=tools,
        tool_choice=None #it's not forcing to use this specific function
    )
    response_message = response.choices[0].message
    
    print ("First response:\n")
    print(response_message)

    # Step 2: check if GPT wanted to call a function
    if response_message.tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message.tool_calls[0].function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message.tool_calls[0].function.arguments)
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )
        
        print("Appending function response to conversation:\n")
        print(function_response)

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "tool",
                "tool_call_id": response_message.tool_calls[0].id,
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        
        print (messages)
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        
        print("Final response:\n")
        return second_response


prompt = input("Ask me about the weather somewhere. You know you want to: ")
print(run_conversation(prompt))